# 지도학습_분류분석_컬럼선택법_클래스불균형처리_모델성능향상시키기

In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import koreanize_matplotlib

In [4]:
data = pd.read_csv("./data/salary2.csv")

In [5]:
data.head(2)

,age,workclass,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,class
0,25,Private,11th,7,Never-married,Machine-op-inspct,Own-child,Black,Male,0,0,40,United-States,<=50K
1,38,Private,HS-grad,9,Married-civ-spouse,Farming-fishing,Husband,White,Male,0,0,50,United-States,<=50K


In [6]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 48842 entries, 0 to 48841
Data columns (total 14 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   age             48842 non-null  int64 
 1   workclass       46043 non-null  object
 2   education       48842 non-null  object
 3   education-num   48842 non-null  int64 
 4   marital-status  48842 non-null  object
 5   occupation      46033 non-null  object
 6   relationship    48842 non-null  object
 7   race            48842 non-null  object
 8   sex             48842 non-null  object
 9   capital-gain    48842 non-null  int64 
 10  capital-loss    48842 non-null  int64 
 11  hours-per-week  48842 non-null  int64 
 12  native-country  47985 non-null  object
 13  class           48842 non-null  object
dtypes: int64(5), object(9)
memory usage: 5.2+ MB


# train/test 분리

In [7]:
from sklearn.model_selection import train_test_split

In [8]:
train, test = train_test_split(data, test_size=0.4, random_state=42, stratify=data['class'])

In [9]:
train.head()

,age,workclass,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,class
47291,18,Private,Some-college,10,Never-married,Transport-moving,Own-child,White,Male,2907,0,30,United-States,<=50K
25800,31,Private,11th,7,Never-married,Craft-repair,Not-in-family,White,Male,0,2001,40,United-States,<=50K
44902,34,Private,Bachelors,13,Never-married,Other-service,Not-in-family,White,Female,0,0,35,United-States,<=50K
41893,59,Private,11th,7,Married-civ-spouse,Adm-clerical,Husband,White,Male,0,0,40,United-States,<=50K
16531,41,Private,Prof-school,15,Married-civ-spouse,Prof-specialty,Wife,White,Female,0,0,40,United-States,>50K


In [10]:
test.head()

,age,workclass,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,class
5127,52,Private,Bachelors,13,Divorced,Craft-repair,Not-in-family,White,Male,0,0,40,United-States,<=50K
34071,44,NaN,Bachelors,13,Married-civ-spouse,NaN,Wife,White,Female,0,0,16,United-States,>50K
29790,41,Private,HS-grad,9,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,40,United-States,<=50K
18182,65,Without-pay,7th-8th,4,Widowed,Farming-fishing,Unmarried,White,Female,0,0,50,United-States,<=50K
42585,57,Private,Some-college,10,Widowed,Tech-support,Not-in-family,White,Female,0,0,16,United-States,<=50K


# 전처리
* 중복제거
* 결측값 삭제
* class 컬럼 label encoding

In [11]:
train = train.drop_duplicates()
test = test.drop_duplicates()

In [13]:
train = train.dropna()
test = test.dropna()

In [14]:
train = train.reset_index(drop=True)
test = test.reset_index(drop=True)

In [15]:
train.head()

,age,workclass,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,class
0,18,Private,Some-college,10,Never-married,Transport-moving,Own-child,White,Male,2907,0,30,United-States,<=50K
1,31,Private,11th,7,Never-married,Craft-repair,Not-in-family,White,Male,0,2001,40,United-States,<=50K
2,34,Private,Bachelors,13,Never-married,Other-service,Not-in-family,White,Female,0,0,35,United-States,<=50K
3,59,Private,11th,7,Married-civ-spouse,Adm-clerical,Husband,White,Male,0,0,40,United-States,<=50K
4,41,Private,Prof-school,15,Married-civ-spouse,Prof-specialty,Wife,White,Female,0,0,40,United-States,>50K


In [16]:
test.head()

,age,workclass,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,class
0,52,Private,Bachelors,13,Divorced,Craft-repair,Not-in-family,White,Male,0,0,40,United-States,<=50K
1,41,Private,HS-grad,9,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,40,United-States,<=50K
2,65,Without-pay,7th-8th,4,Widowed,Farming-fishing,Unmarried,White,Female,0,0,50,United-States,<=50K
3,57,Private,Some-college,10,Widowed,Tech-support,Not-in-family,White,Female,0,0,16,United-States,<=50K
4,24,Private,Bachelors,13,Never-married,Tech-support,Not-in-family,White,Female,3674,0,40,United-States,<=50K


In [17]:
from sklearn.preprocessing import LabelEncoder

In [18]:
le = LabelEncoder()
le.fit(train['class'])
train['class'] = le.transform(train['class'])
test['class'] = le.transform(test['class'])

In [19]:
train.head()

,age,workclass,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,class
0,18,Private,Some-college,10,Never-married,Transport-moving,Own-child,White,Male,2907,0,30,United-States,0
1,31,Private,11th,7,Never-married,Craft-repair,Not-in-family,White,Male,0,2001,40,United-States,0
2,34,Private,Bachelors,13,Never-married,Other-service,Not-in-family,White,Female,0,0,35,United-States,0
3,59,Private,11th,7,Married-civ-spouse,Adm-clerical,Husband,White,Male,0,0,40,United-States,0
4,41,Private,Prof-school,15,Married-civ-spouse,Prof-specialty,Wife,White,Female,0,0,40,United-States,1


In [20]:
test.head()

,age,workclass,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,class
0,52,Private,Bachelors,13,Divorced,Craft-repair,Not-in-family,White,Male,0,0,40,United-States,0
1,41,Private,HS-grad,9,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,40,United-States,0
2,65,Without-pay,7th-8th,4,Widowed,Farming-fishing,Unmarried,White,Female,0,0,50,United-States,0
3,57,Private,Some-college,10,Widowed,Tech-support,Not-in-family,White,Female,0,0,16,United-States,0
4,24,Private,Bachelors,13,Never-married,Tech-support,Not-in-family,White,Female,3674,0,40,United-States,0


# 중요 컬럼 선택하기
* EDA를 통해서 종속변수와 중요한 관계가 있는 변수들만 선택
* 수치형 변수는 상관분석 결과를 통해 선택, 범주형 변수는 카이제곱 통계량 분석을 통해 선택
* 머신러닝 알고리즘을 통한 1차 분석 후 중요하게 사용된 변수만 선택

# tree 계열 모델의 feature_importance로 선택하기

In [25]:
X_train = train.drop('class', axis=1)
y_train = train['class']
display(X_train.head())
y_train.value_counts()

,age,workclass,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country
0,18,Private,Some-college,10,Never-married,Transport-moving,Own-child,White,Male,2907,0,30,United-States
1,31,Private,11th,7,Never-married,Craft-repair,Not-in-family,White,Male,0,2001,40,United-States
2,34,Private,Bachelors,13,Never-married,Other-service,Not-in-family,White,Female,0,0,35,United-States
3,59,Private,11th,7,Married-civ-spouse,Adm-clerical,Husband,White,Male,0,0,40,United-States
4,41,Private,Prof-school,15,Married-civ-spouse,Prof-specialty,Wife,White,Female,0,0,40,United-States


class
0    18325
1     6185
Name: count, dtype: int64

In [26]:
X_test = test.drop('class', axis=1)
y_test = test['class']
display(X_test.head())
y_test.value_counts()

,age,workclass,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country
0,52,Private,Bachelors,13,Divorced,Craft-repair,Not-in-family,White,Male,0,0,40,United-States
1,41,Private,HS-grad,9,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,40,United-States
2,65,Without-pay,7th-8th,4,Widowed,Farming-fishing,Unmarried,White,Female,0,0,50,United-States
3,57,Private,Some-college,10,Widowed,Tech-support,Not-in-family,White,Female,0,0,16,United-States
4,24,Private,Bachelors,13,Never-married,Tech-support,Not-in-family,White,Female,3674,0,40,United-States


class
0    12463
1     4201
Name: count, dtype: int64

In [21]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder

In [23]:
def to_ohe(X_train, X_test):
    cat_cols = X_train.select_dtypes(include='object').columns
    ct = ColumnTransformer(transformers=[
        ('ohe', OneHotEncoder(drop='first', handle_unknown='ignore', sparse_output=False),
         cat_cols)], remainder='passthrough')
    ct.set_output(transform='pandas')
    X_train = ct.fit_transform(X_train)
    X_test = ct.transform(X_test)
    return X_train, X_test
    

In [27]:
X_train, X_test = to_ohe(X_train, X_test)
display(X_train.head())
display(X_test.head())

,ohe__workclass_ Local-gov,ohe__workclass_ Private,ohe__workclass_ Self-emp-inc,ohe__workclass_ Self-emp-not-inc,ohe__workclass_ State-gov,ohe__workclass_ Without-pay,ohe__education_ 11th,ohe__education_ 12th,ohe__education_ 1st-4th,ohe__education_ 5th-6th,...,ohe__native-country_ Thailand,ohe__native-country_ Trinadad&Tobago,ohe__native-country_ United-States,ohe__native-country_ Vietnam,ohe__native-country_ Yugoslavia,remainder__age,remainder__education-num,remainder__capital-gain,remainder__capital-loss,remainder__hours-per-week
0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,18,10,2907,0,30
1,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,31,7,0,2001,40
2,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,34,13,0,0,35
3,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,59,7,0,0,40
4,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,41,15,0,0,40


,ohe__workclass_ Local-gov,ohe__workclass_ Private,ohe__workclass_ Self-emp-inc,ohe__workclass_ Self-emp-not-inc,ohe__workclass_ State-gov,ohe__workclass_ Without-pay,ohe__education_ 11th,ohe__education_ 12th,ohe__education_ 1st-4th,ohe__education_ 5th-6th,...,ohe__native-country_ Thailand,ohe__native-country_ Trinadad&Tobago,ohe__native-country_ United-States,ohe__native-country_ Vietnam,ohe__native-country_ Yugoslavia,remainder__age,remainder__education-num,remainder__capital-gain,remainder__capital-loss,remainder__hours-per-week
0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,52,13,0,0,40
1,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,41,9,0,0,40
2,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,65,4,0,0,50
3,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,57,10,0,0,16
4,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,24,13,3674,0,40


# 의사결정나무 분석 후 feature_importance 추출

In [29]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score, classification_report

In [30]:
dtc = DecisionTreeClassifier(max_depth=4, random_state=10)
dtc.fit(X_train, y_train)
pred1 = dtc.predict(X_test)
print(accuracy_score(y_test, pred1))
print(classification_report(y_test, pred1))

0.8380940950552088
              precision    recall  f1-score   support

           0       0.86      0.94      0.90     12463
           1       0.75      0.54      0.63      4201

    accuracy                           0.84     16664
   macro avg       0.80      0.74      0.76     16664
weighted avg       0.83      0.84      0.83     16664



In [35]:
dtc_importance = pd.DataFrame(dtc.feature_importances_, index=dtc.feature_names_in_, columns=['importance'])
dtc_importance.sort_values(by='importance', ascending=False).head(20)

,importance
ohe__marital-status_ Married-civ-spouse,0.502716
remainder__capital-gain,0.237002
remainder__education-num,0.220531
remainder__capital-loss,0.026382
remainder__hours-per-week,0.011622
remainder__age,0.001748
ohe__workclass_ Private,0.000000
ohe__education_ 12th,0.000000
ohe__education_ 1st-4th,0.000000
ohe__education_ 5th-6th,0.000000


# 의사결정나무에서 중요하게 본 변수만 선택해서 분석

In [39]:
dtc_train = train[['age', 'education-num', 'marital-status', 'capital-gain', 'capital-loss', 'hours-per-week', 'class']]

In [40]:
dtc_train

,age,education-num,marital-status,capital-gain,capital-loss,hours-per-week,class
0,18,10,Never-married,2907,0,30,0
1,31,7,Never-married,0,2001,40,0
2,34,13,Never-married,0,0,35,0
3,59,7,Married-civ-spouse,0,0,40,0
4,41,15,Married-civ-spouse,0,0,40,1
...,...,...,...,...,...,...,...
24505,66,4,Divorced,0,0,40,0
24506,48,13,Married-civ-spouse,0,0,40,0
24507,42,10,Divorced,0,0,40,0
24508,47,14,Never-married,0,0,35,1


In [42]:
dtc_train.columns

Index(['age', 'education-num', 'marital-status', 'capital-gain',
       'capital-loss', 'hours-per-week', 'class'],
      dtype='object')

In [43]:
dtc_test = test[dtc_train.columns]
dtc_test

,age,education-num,marital-status,capital-gain,capital-loss,hours-per-week,class
0,52,13,Divorced,0,0,40,0
1,41,9,Married-civ-spouse,0,0,40,0
2,65,4,Widowed,0,0,50,0
3,57,10,Widowed,0,0,16,0
4,24,13,Never-married,3674,0,40,0
...,...,...,...,...,...,...,...
16659,33,9,Divorced,0,0,20,0
16660,29,13,Never-married,4650,0,40,0
16661,51,13,Married-civ-spouse,0,1672,70,0
16662,34,13,Married-civ-spouse,7298,0,60,1


In [45]:
dtc_X_train = dtc_train.drop('class', axis=1)
dtc_y_train = dtc_train['class']
dtc_X_test = dtc_test.drop('class', axis=1)
dtc_y_test = dtc_test['class']

In [46]:
dtc_X_train.head()

,age,education-num,marital-status,capital-gain,capital-loss,hours-per-week
0,18,10,Never-married,2907,0,30
1,31,7,Never-married,0,2001,40
2,34,13,Never-married,0,0,35
3,59,7,Married-civ-spouse,0,0,40
4,41,15,Married-civ-spouse,0,0,40


In [49]:
dtc_y_train.head()

0    0
1    0
2    0
3    0
4    1
Name: class, dtype: int64

In [47]:
dtc_X_test.head()

,age,education-num,marital-status,capital-gain,capital-loss,hours-per-week
0,52,13,Divorced,0,0,40
1,41,9,Married-civ-spouse,0,0,40
2,65,4,Widowed,0,0,50
3,57,10,Widowed,0,0,16
4,24,13,Never-married,3674,0,40


In [50]:
dtc_y_test.head()

0    0
1    0
2    0
3    0
4    0
Name: class, dtype: int64

In [52]:
dtc_X_train, dtc_X_test = to_ohe(dtc_X_train, dtc_X_test)
display(dtc_X_train)
display(dtc_X_test)

,ohe__marital-status_ Married-AF-spouse,ohe__marital-status_ Married-civ-spouse,ohe__marital-status_ Married-spouse-absent,ohe__marital-status_ Never-married,ohe__marital-status_ Separated,ohe__marital-status_ Widowed,remainder__age,remainder__education-num,remainder__capital-gain,remainder__capital-loss,remainder__hours-per-week
0,0.0,0.0,0.0,1.0,0.0,0.0,18,10,2907,0,30
1,0.0,0.0,0.0,1.0,0.0,0.0,31,7,0,2001,40
2,0.0,0.0,0.0,1.0,0.0,0.0,34,13,0,0,35
3,0.0,1.0,0.0,0.0,0.0,0.0,59,7,0,0,40
4,0.0,1.0,0.0,0.0,0.0,0.0,41,15,0,0,40
...,...,...,...,...,...,...,...,...,...,...,...
24505,0.0,0.0,0.0,0.0,0.0,0.0,66,4,0,0,40
24506,0.0,1.0,0.0,0.0,0.0,0.0,48,13,0,0,40
24507,0.0,0.0,0.0,0.0,0.0,0.0,42,10,0,0,40
24508,0.0,0.0,0.0,1.0,0.0,0.0,47,14,0,0,35


,ohe__marital-status_ Married-AF-spouse,ohe__marital-status_ Married-civ-spouse,ohe__marital-status_ Married-spouse-absent,ohe__marital-status_ Never-married,ohe__marital-status_ Separated,ohe__marital-status_ Widowed,remainder__age,remainder__education-num,remainder__capital-gain,remainder__capital-loss,remainder__hours-per-week
0,0.0,0.0,0.0,0.0,0.0,0.0,52,13,0,0,40
1,0.0,1.0,0.0,0.0,0.0,0.0,41,9,0,0,40
2,0.0,0.0,0.0,0.0,0.0,1.0,65,4,0,0,50
3,0.0,0.0,0.0,0.0,0.0,1.0,57,10,0,0,16
4,0.0,0.0,0.0,1.0,0.0,0.0,24,13,3674,0,40
...,...,...,...,...,...,...,...,...,...,...,...
16659,0.0,0.0,0.0,0.0,0.0,0.0,33,9,0,0,20
16660,0.0,0.0,0.0,1.0,0.0,0.0,29,13,4650,0,40
16661,0.0,1.0,0.0,0.0,0.0,0.0,51,13,0,1672,70
16662,0.0,1.0,0.0,0.0,0.0,0.0,34,13,7298,0,60


In [53]:
dtc2 = DecisionTreeClassifier(max_depth=4, random_state=10)
dtc2.fit(dtc_X_train, dtc_y_train)
pred2 = dtc2.predict(dtc_X_test)
print(accuracy_score(dtc_y_test, pred2))
print(classification_report(dtc_y_test, pred2))

0.8380940950552088
              precision    recall  f1-score   support

           0       0.86      0.94      0.90     12463
           1       0.75      0.54      0.63      4201

    accuracy                           0.84     16664
   macro avg       0.80      0.74      0.76     16664
weighted avg       0.83      0.84      0.83     16664



# randomforest의 경우

In [54]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report

In [56]:
rfc = RandomForestClassifier(max_depth=4, n_jobs=-1,random_state=10)
rfc.fit(X_train, y_train)
pred1 = rfc.predict(X_test)
print(accuracy_score(y_test, pred1))
print(classification_report(y_test, pred1))

0.8127700432069132
              precision    recall  f1-score   support

           0       0.80      1.00      0.89     12463
           1       0.95      0.27      0.42      4201

    accuracy                           0.81     16664
   macro avg       0.88      0.63      0.65     16664
weighted avg       0.84      0.81      0.77     16664



In [59]:
rfc_importance = pd.DataFrame(rfc.feature_importances_, index=rfc.feature_names_in_, columns=['importance'])
rfc_importance.sort_values(by='importance', ascending=False).head(50)

,importance
ohe__marital-status_ Married-civ-spouse,0.228391
remainder__capital-gain,0.184174
remainder__education-num,0.114874
ohe__marital-status_ Never-married,0.086539
remainder__age,0.069498
remainder__capital-loss,0.041603
ohe__relationship_ Not-in-family,0.035654
ohe__occupation_ Exec-managerial,0.031593
ohe__sex_ Male,0.030942
ohe__relationship_ Own-child,0.029844


# XGBOOST의 경우

In [60]:
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score, classification_report

In [61]:
xgb = XGBClassifier(max_depth=4, n_jobs=-1,random_state=10)
xgb.fit(X_train, y_train)
pred2 = xgb.predict(X_test)
print(accuracy_score(y_test, pred2))
print(classification_report(y_test, pred2))

0.8701992318771004
              precision    recall  f1-score   support

           0       0.89      0.94      0.92     12463
           1       0.78      0.67      0.72      4201

    accuracy                           0.87     16664
   macro avg       0.84      0.80      0.82     16664
weighted avg       0.87      0.87      0.87     16664



In [62]:
xgb_importance = pd.DataFrame(xgb.feature_importances_, index=xgb.feature_names_in_, columns=['importance'])
xgb_importance.sort_values(by='importance', ascending=False).head(50)

,importance
ohe__marital-status_ Married-civ-spouse,0.383847
remainder__education-num,0.084336
remainder__capital-gain,0.048743
ohe__occupation_ Exec-managerial,0.045218
ohe__occupation_ Other-service,0.041056
ohe__occupation_ Farming-fishing,0.029075
remainder__capital-loss,0.026672
ohe__relationship_ Own-child,0.020889
ohe__occupation_ Handlers-cleaners,0.018728
remainder__age,0.015096


# 클래스 불균형을 처리하는 방법